In [218]:
import neurokit2 as nk
import numpy as np
import pandas as pd
import tqdm
import matplotlib.pyplot as plt
from biosppy.signals import ecg
from scipy import fftpack
from sklearn.preprocessing import normalize

In [2]:
X_train = pd.read_csv("X_train.csv").drop(columns=["id"])

In [215]:
info = ecg.ecg(pat, sampling_rate =300, show=False)
np.mean(info[4]

array([[ 5.42564934e+01,  5.06679232e+01,  4.75045855e+01, ...,
         3.38392757e+01,  2.58068737e+01,  1.52189267e+01],
       [-2.99455530e+01, -2.42649288e+01, -1.95512517e+01, ...,
        -3.69083834e+01, -2.13244977e+01, -3.76182104e+00],
       [-3.73806647e+00, -1.89161884e+00, -1.40955712e-01, ...,
         4.90869044e-01,  7.05302431e+00,  1.43398545e+01],
       ...,
       [-2.00746191e+01, -2.06669274e+01, -2.09863976e+01, ...,
        -2.02362824e+01, -1.41537120e+01, -6.69066205e+00],
       [-2.25612959e+01, -2.07742808e+01, -1.69330635e+01, ...,
         5.28747507e+00,  5.10407815e+00,  4.80550491e+00],
       [-1.04779096e+01, -7.53463269e+00, -4.99859366e+00, ...,
        -4.64357806e-02, -2.83947056e+00, -5.74631297e+00]])

In [324]:
def feature_extraction(pat):
    #extract peaks and other useful information usign biosppy and neurokit2
    info = ecg.ecg(pat, sampling_rate =300, show=False)
    _, rpeaks = nk.ecg_peaks(info['filtered'], sampling_rate=300, correct_artifacts=False)
    peaks = np.copy(rpeaks['ECG_R_Peaks'])[0:-1] # las value led to index error
    signal, wave = nk.ecg_delineate(info['filtered'], peaks, sampling_rate=300)
    epochs = nk.ecg_segment(signal, rpeaks=rpeaks["ECG_R_Peaks"], sampling_rate=300, show=False)
    hrv_indices = nk.hrv_time(rpeaks["ECG_R_Peaks"], sampling_rate=300, show=False)
    
    # transoform wave item values to numpy arrays
    for key,value in wave.items():
        value= np.array(value)
        wave[str(key)] = np.array(value[~np.isnan(value)],dtype=int)
        
    # calculate number of indices between rpeaks
    rPeakIntervals = []
    for a in range(0, len(rpeaks["ECG_R_Peaks"])-1):
        rPeakIntervals.append(rpeaks["ECG_R_Peaks"][a+1]-rpeaks["ECG_R_Peaks"][a])
    rPeakIntervals = np.array(rPeakIntervals)
    
    rPeakIntervals = []
    for a in range(0, len(rpeaks["ECG_R_Peaks"])-1):
            rPeakIntervals.append(rpeaks["ECG_R_Peaks"][a+1]-rpeaks["ECG_R_Peaks"][a])
    rPeakIntervals = np.array(rPeakIntervals)
        
    qrsInterval = []
    for b in range(0,len(wave["ECG_S_Peaks"])):
        qrsInterval.append(wave["ECG_S_Peaks"][b]-rpeaks["ECG_R_Peaks"][b])
    qrsInterval = np.array(qrsInterval)
    
    qrInterval = []
    for e in range(0,len(wave["ECG_Q_Peaks"])):
        qrInterval.append(rpeaks["ECG_R_Peaks"][e]-wave["ECG_Q_Peaks"][e])
    qrInterval = np.array(qrInterval)
    
    rsInterval = []
    for f in range(0,len(wave["ECG_S_Peaks"])):
        rsInterval.append(wave["ECG_S_Peaks"][b]-rpeaks["ECG_R_Peaks"][b])
    rsInterval = np.array(rsInterval)
    
    pInterval = []
    for c in range(1,len(wave["ECG_P_Peaks"])):
        pInterval.append(wave["ECG_P_Peaks"][c]-wave["ECG_P_Peaks"][c-1])
    pInterval = np.array(pInterval)
    
    prInterval = []
    for d in range(0,len(wave["ECG_P_Peaks"])):
        prInterval.append(rpeaks["ECG_R_Peaks"][d]-wave["ECG_P_Peaks"][d])
    prInterval = np.array(prInterval)
    
    #quality meassure 
    quality = nk.ecg_quality(info['filtered'], sampling_rate=300)
    #something like heart rate
    ecg_rate = nk.signal_rate(_, sampling_rate=300, desired_length=len(_))
    #not realy heart rate
    heartRate = nk.ecg_rsp(ecg_rate, sampling_rate=300)
    
    HRV_MeanNN = hrv_indices['HRV_MeanNN'][0]
    HRV_SDNN = hrv_indices['HRV_SDNN'][0]
    HRV_RMSSD = hrv_indices['HRV_RMSSD'][0]
    HRV_SDSD = hrv_indices['HRV_SDSD'][0]
    
    #means for all intervals
    rPeakIntervalMean = np.nanmean(rPeakIntervals)
    qrsIntervalMean = np.nanmean(qrsInterval)
    qrIntervalMean = np.nanmean(qrInterval)
    rsIntervalMean = np.nanmean(rsInterval)
    pIntervalMean = np.nanmean(pInterval)
    prIntervalMean = np.nanmean(prInterval)
    
    #std deviation for all intervals
    rPeakIntervalStd = np.nanstd(rPeakIntervals)
    qrsIntervalStd = np.nanstd(qrsInterval)
    qrIntervalStd = np.nanstd(qrInterval)
    rsIntervalStd = np.nanstd(rsInterval)
    pIntervalStd = np.nanstd(pInterval)
    prIntervalStd = np.nanstd(prInterval)

    #Max for all intervals
    rPeakIntervalMax = np.nanmax(rPeakIntervals)
    qrsIntervalMax = np.nanmax(qrsInterval)
    qrIntervalMax = np.nanmax(qrInterval)
    rsIntervalMax = np.nanmax(rsInterval)
    pIntervalMax = np.nanmax(pInterval)
    prIntervalMax = np.nanmax(prInterval)
    
    #Min for all intervals
    rPeakIntervalMin = np.nanmin(rPeakIntervals)
    qrsIntervalMin = np.nanmin(qrsInterval)
    qrIntervalMin = np.nanmin(qrInterval)
    rsIntervalMin = np.nanmin(rsInterval)
    pIntervalMin = np.nanmin(pInterval)
    prIntervalMin = np.nanmin(prInterval)
    
    #mean of heartbeat hight
    pPeaksMean = np.nanmean([info['filtered'][l] for l in wave["ECG_P_Peaks"]])
    tPeaksMean = np.nanmean([info['filtered'][l] for l in wave["ECG_T_Peaks"]])
    rPeaksMean = np.nanmean([info['filtered'][l] for l in rpeaks["ECG_R_Peaks"]])
    qPeaksMean = np.nanmean([info['filtered'][l] for l in wave["ECG_Q_Peaks"]])
    sPeaksMean = np.nanmean([info['filtered'][l] for l in wave["ECG_S_Peaks"]])
    sPeaksMin = np.nanmin([info['filtered'][l] for l in wave["ECG_S_Peaks"]])
    tOffsetMean = np.nanmean([info['filtered'][l] for l in wave["ECG_T_Offsets"]])
    pOnsetMean = np.nanmean([info['filtered'][l] for l in wave["ECG_P_Onsets"]])
    
    #std of hearbeat hight
    pPeaksStd = np.nanstd([info['filtered'][l] for l in wave["ECG_P_Peaks"]])
    tPeaksStd = np.nanstd([info['filtered'][l] for l in wave["ECG_T_Peaks"]])
    rPeaksStd = np.nanstd([info['filtered'][l] for l in rpeaks["ECG_R_Peaks"]])
    qPeaksStd = np.nanstd([info['filtered'][l] for l in wave["ECG_Q_Peaks"]])
    sPeaksStd = np.nanstd([info['filtered'][l] for l in wave["ECG_S_Peaks"]])
    tOffsetStd = np.nanstd([info['filtered'][l] for l in wave["ECG_T_Offsets"]])
    pOnsetStd = np.nanstd([info['filtered'][l] for l in wave["ECG_P_Onsets"]])
    
    #summed variance over the overlayed heartbeats
    summed_variance_heartbeat = np.sum(np.std(normalize(info[4]), axis=0))
    
    
    heartRateMean = np.nanmean(heartRate)
    qualityMean = np.nanmean(quality)
    ecgRateMean = np.nanmean(ecg_rate)
    
    heartRateStd = np.nanstd(heartRate)
    qualityStd = np.nanstd(quality)
    ecgRateStd = np.nanstd(ecg_rate)
    
    
    
    
    features = []
    
    features.append([HRV_MeanNN,
                         HRV_SDNN,
                         HRV_RMSSD,
                         HRV_SDSD,
                         rPeakIntervalMean,
                         qrsIntervalMean,
                         qrIntervalMean,
                         rsIntervalMean,
                         pIntervalMean,
                         prIntervalMean,
                         pPeaksMean, 
                         tPeaksMean,
                         rPeaksMean, 
                         qPeaksMean,
                         sPeaksMean,
                         sPeaksMin,
                         tOffsetMean,
                         pOnsetMean,
                         heartRateMean,
                         qualityMean,
                         rPeakIntervalStd,
                         qrsIntervalStd,
                         qrIntervalStd,
                         rsIntervalStd,
                         pIntervalStd,
                         prIntervalStd,
                         pPeaksStd,
                         tPeaksStd,
                         rPeaksStd,
                         qPeaksStd,
                         sPeaksStd,
                         tOffsetStd,
                         heartRateStd,
                         qualityStd,
                         rPeakIntervalMax,
                         qrsIntervalMax,
                         qrIntervalMax,
                         rsIntervalMax,
                         pIntervalMax,
                         prIntervalMax,
                         rPeakIntervalMin,
                         qrsIntervalMin,
                         qrIntervalMin,
                         rsIntervalMin,
                         pIntervalMin,
                         prIntervalMin,
                         summed_variance_heartbeat,
                         np.log(.001+np.abs(rPeakIntervalMean)),
                         np.log(.001+np.abs(qrsIntervalMean)),
                         np.log(.001+np.abs(pPeaksMean)),
                         np.log(.001+np.abs(tPeaksMean)),
                         np.log(.001+np.abs(qPeaksMean)),
                         np.log(.001+np.abs(sPeaksMean)),
                         np.log(.001+np.abs(tOffsetMean)),
                         np.log(.001+np.abs(pOnsetMean)),
                         np.log(.001+rPeakIntervalStd),
                         np.log(.001+qrsIntervalStd),
                         np.log(.001+pPeaksStd),
                         np.log(.001+tPeaksStd),
                         np.log(.001+rPeaksStd),
                         np.log(.001+qPeaksStd),
                         np.log(.001+sPeaksStd),
                         np.log(.001+tOffsetStd),
                         pOnsetStd])

    return np.array(features)

In [367]:
#build feature vector
pat0 = X_train.loc[0].dropna().to_numpy(dtype='float32')
feature_vec = feature_extraction(pat0)

for i in tqdm.trange(1, X_train.shape[0]):
    pat_i = X_train.loc[i].dropna().to_numpy(dtype='float32')
    pat_i, is_inverted = nk.ecg_invert(pat_i, sampling_rate=300, show=False)
    feature_i = feature_extraction(pat_i)
    feature_vec = np.concatenate((feature_vec, feature_i))

  0%|▏                                                            | 14/5116 [00:02<17:34,  4.84it/s]/Users/paul/miniforge3/envs/tensorflow/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/paul/miniforge3/envs/tensorflow/lib/python3.9/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
  0%|▎                                                            | 24/5116 [00:04<16:14,  5.23it/s]/Users/paul/miniforge3/envs/tensorflow/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/paul/miniforge3/envs/tensorflow/lib/python3.9/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
  1%|▍                                    

  6%|███▊                                                        | 323/5116 [01:09<14:42,  5.43it/s]/Users/paul/miniforge3/envs/tensorflow/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/paul/miniforge3/envs/tensorflow/lib/python3.9/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
  7%|███▉                                                        | 338/5116 [01:12<15:02,  5.29it/s]/Users/paul/miniforge3/envs/tensorflow/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/paul/miniforge3/envs/tensorflow/lib/python3.9/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
  7%|████▎                                

 14%|████████▌                                                   | 727/5116 [02:36<16:33,  4.42it/s]/Users/paul/miniforge3/envs/tensorflow/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/paul/miniforge3/envs/tensorflow/lib/python3.9/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
 14%|████████▌                                                   | 730/5116 [02:37<19:35,  3.73it/s]/Users/paul/miniforge3/envs/tensorflow/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/paul/miniforge3/envs/tensorflow/lib/python3.9/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
 14%|████████▋                            

 20%|███████████▌                                               | 1007/5116 [03:37<13:48,  4.96it/s]/Users/paul/miniforge3/envs/tensorflow/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/paul/miniforge3/envs/tensorflow/lib/python3.9/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
 21%|████████████▍                                              | 1076/5116 [03:52<14:30,  4.64it/s]/Users/paul/miniforge3/envs/tensorflow/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/paul/miniforge3/envs/tensorflow/lib/python3.9/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
 21%|████████████▌                        

 29%|█████████████████▏                                         | 1495/5116 [05:23<09:56,  6.07it/s]/Users/paul/miniforge3/envs/tensorflow/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/paul/miniforge3/envs/tensorflow/lib/python3.9/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
 30%|█████████████████▍                                         | 1510/5116 [05:26<14:25,  4.17it/s]/Users/paul/miniforge3/envs/tensorflow/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/paul/miniforge3/envs/tensorflow/lib/python3.9/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
 30%|█████████████████▋                   

...

 71%|█████████████████████████████████████████▊                 | 3625/5116 [12:45<05:01,  4.94it/s]/Users/paul/miniforge3/envs/tensorflow/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/paul/miniforge3/envs/tensorflow/lib/python3.9/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
 72%|██████████████████████████████████████████▋                | 3697/5116 [13:00<04:42,  5.02it/s]/Users/paul/miniforge3/envs/tensorflow/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/paul/miniforge3/envs/tensorflow/lib/python3.9/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
 73%|█████████████████████████████████████

 77%|█████████████████████████████████████████████▍             | 3936/5116 [13:50<04:25,  4.44it/s]/Users/paul/miniforge3/envs/tensorflow/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/paul/miniforge3/envs/tensorflow/lib/python3.9/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
 77%|█████████████████████████████████████████████▋             | 3961/5116 [13:56<05:07,  3.75it/s]/Users/paul/miniforge3/envs/tensorflow/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/paul/miniforge3/envs/tensorflow/lib/python3.9/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
 78%|█████████████████████████████████████

 83%|████████████████████████████████████████████████▉          | 4243/5116 [14:54<02:41,  5.41it/s]/Users/paul/miniforge3/envs/tensorflow/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/paul/miniforge3/envs/tensorflow/lib/python3.9/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
 84%|█████████████████████████████████████████████████▍         | 4284/5116 [15:02<02:57,  4.69it/s]/Users/paul/miniforge3/envs/tensorflow/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/paul/miniforge3/envs/tensorflow/lib/python3.9/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
 85%|█████████████████████████████████████

 89%|████████████████████████████████████████████████████▎      | 4535/5116 [15:52<01:46,  5.46it/s]/Users/paul/miniforge3/envs/tensorflow/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/paul/miniforge3/envs/tensorflow/lib/python3.9/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
 89%|████████████████████████████████████████████████████▍      | 4548/5116 [15:55<02:32,  3.72it/s]/Users/paul/miniforge3/envs/tensorflow/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/paul/miniforge3/envs/tensorflow/lib/python3.9/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
 89%|█████████████████████████████████████

 94%|███████████████████████████████████████████████████████▎   | 4797/5116 [16:45<01:03,  5.01it/s]/Users/paul/miniforge3/envs/tensorflow/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/paul/miniforge3/envs/tensorflow/lib/python3.9/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
 94%|███████████████████████████████████████████████████████▋   | 4829/5116 [16:51<00:48,  5.86it/s]/Users/paul/miniforge3/envs/tensorflow/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/paul/miniforge3/envs/tensorflow/lib/python3.9/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
 95%|█████████████████████████████████████

In [368]:
feature_vec.shape

(5117, 64)

In [369]:
df_features = pd.DataFrame(feature_vec)

In [370]:
df_features

,0,1,2,3,4,5,6,7,8,9,...,54,55,56,57,58,59,60,61,62,63
0,815.909091,64.985983,63.715446,63.760285,244.772727,13.212121,10.075758,8.0,245.246154,45.651515,...,2.104580,2.962617,1.468057,3.297508,3.874312,3.891545,2.812233,3.135966,3.405693,19.288813
1,788.380952,54.240878,61.530055,62.437531,236.514286,8.228571,12.485714,7.0,237.441176,51.742857,...,2.074463,2.775031,0.971640,3.092029,3.707863,3.550458,2.752231,3.686132,3.778790,18.467852
2,888.000000,77.901598,66.629300,66.264592,266.400000,6.866667,11.066667,7.0,270.137931,42.333333,...,1.684761,3.134566,-1.076065,2.615898,2.591317,4.992898,2.433126,2.766482,2.729964,9.330390
3,411.111111,73.383821,99.593947,100.330495,123.333333,32.144928,6.884058,14.0,185.400000,-1233.760870,...,3.005885,3.084477,2.914198,3.285937,3.055057,3.905623,3.887050,3.634371,3.862021,74.224586
4,612.074074,79.335271,105.995759,107.221157,183.622222,14.377778,6.488889,13.0,188.883721,-108.636364,...,3.425364,3.158516,1.224432,2.714311,3.784912,3.974346,4.401440,3.992879,3.507524,31.642543
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5112,692.028986,209.250078,234.104785,239.207043,207.608696,10.869565,10.739130,28.0,207.181818,65.086957,...,4.692946,4.117348,2.305851,5.013909,5.310535,5.371113,5.223909,5.102073,5.045779,157.553201
5113,864.623656,14.393348,10.954451,11.058881,259.387097,8.967742,10.612903,7.0,258.833333,38.935484,...,2.615959,1.446634,1.193303,2.400577,2.504654,3.200025,2.707966,2.968458,2.522019,10.581359
5114,788.857143,95.352606,32.584731,32.965439,236.657143,7.200000,10.371429,7.0,236.470588,41.685714,...,2.159531,3.339151,-0.644885,3.642163,2.664011,3.738771,3.336557,3.370450,3.060632,15.068207
5115,848.333333,27.965673,34.485465,35.044604,254.500000,8.531250,10.906250,8.0,254.677419,47.375000,...,1.866791,2.111252,-0.693102,2.114114,2.172123,2.969135,2.326365,2.926380,3.203742,8.135307


In [371]:
df_features.to_csv('features_ata.csv')